<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/ALIGNN_FF_DB_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Search if an example system such as AgAu was included in the ALIGNN-FF database during training

In [1]:
import os
if not os.path.exists('jarvis_leaderboard'):
  !git clone https://github.com/usnistgov/jarvis_leaderboard.git
os.chdir('jarvis_leaderboard')
!pip install -e .

Cloning into 'jarvis_leaderboard'...
remote: Enumerating objects: 35441, done.
remote: Counting objects: 100% (4241/4241), done.
remote: Compressing objects: 100% (586/586), done.
remote: Total 35441 (delta 2148), reused 3979 (delta 2014), pack-reused 31200
Receiving objects: 100% (35441/35441), 280.03 MiB | 29.17 MiB/s, done.
Resolving deltas: 100% (18316/18316), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/jarvis_leaderboard
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 43.8 MB/s eta 0:00:00

In [2]:
from jarvis_leaderboard.rebuild import get_metric_value,get_results

In [3]:
!ls jarvis_leaderboard/contributions/alignnff_wt10/AI-MLFF-energy-alignn_ff_db-test-mae.csv.zip

jarvis_leaderboard/contributions/alignnff_wt10/AI-MLFF-energy-alignn_ff_db-test-mae.csv.zip


In [4]:
res = get_metric_value(csv_path='jarvis_leaderboard/contributions/alignnff_wt10/AI-MLFF-energy-alignn_ff_db-test-mae.csv.zip')

In [5]:
res

{'category': 'AI',
 'subcat': 'MLFF',
 'dataset': 'alignn_ff_db',
 'prop': 'energy',
 'data_split': 'test',
 'csv_path': 'jarvis_leaderboard/contributions/alignnff_wt10/AI-MLFF-energy-alignn_ff_db-test-mae.csv.zip',
 'metric': 'mae',
 'model_name': 'ALIGNN',
 'team_name': 'JARVIS',
 'date_submitted': '01-14-2023',
 'project_url': 'https://pubs.rsc.org/en/content/articlehtml/2023/dd/d2dd00096b',
 'num_entries': 15355,
 'github_url': 'https://github.com/usnistgov/jarvis_leaderboard/tree/main/jarvis_leaderboard/contributions/alignnff_wt10',
 'dataset_size': 307111,
 'res': 0.097,
 'df':                          id    target  prediction    actual
 0      JVASP-140261_main-15 -2.689496   -2.718657 -2.689496
 1       JVASP-25777_main-40 -6.915743   -6.846940 -6.915743
 2      JVASP-140580_main-58 -3.717568   -3.731820 -3.717568
 3       JVASP-116078_main-0 -1.254895   -0.931186 -1.254895
 4       JVASP-115570_main-0 -2.414957   -2.456279 -2.414957
 ...                     ...       ...      

In [6]:
from jarvis.db.figshare import data

alignn_ff_db = data('alignn_ff_db')

Obtaining ALIGNN-FF training DB 300k ...
Reference:https://arxiv.org/abs/2209.05554


100%|██████████| 72.5M/72.5M [00:03<00:00, 23.2MiB/s]


Loading the zipfile...
Loading completed.


In [7]:
import pandas as pd
df = pd.DataFrame(alignn_ff_db)

In [8]:
len(res['df']['id'])

15355

In [9]:
import zipfile
import json
json_zip = 'jarvis_leaderboard/benchmarks/AI/MLFF/alignn_ff_db_energy.json.zip'
zp = zipfile.ZipFile(json_zip)
temp = 'alignn_ff_db_energy.json'
train_val_test = json.loads(zp.read(temp))


In [10]:
train_ids = list(train_val_test['train'].keys())

In [11]:
len(train_ids)

276401

Check all the test split ids exist in the ALIGNN-FF-DB

In [12]:
train_df = df[df['jid'].isin(train_ids)]

In [13]:
from tqdm import tqdm
tqdm.pandas()
from jarvis.core.atoms import Atoms
train_df['search_string'] = train_df['atoms'].progress_apply(lambda x:(Atoms.from_dict(x)).composition.search_string)

100%|██████████| 276401/276401 [00:09<00:00, 28905.76it/s]
<ipython-input-13-af85d142d279>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['search_string'] = train_df['atoms'].progress_apply(lambda x:(Atoms.from_dict(x)).composition.search_string)


In [14]:
from jarvis.core.composition import Composition
my_search_string = Composition.from_string('AgAu').search_string
filtered_df = train_df[train_df['search_string']==my_search_string]

In [15]:
len(filtered_df)

56

In [16]:
print (filtered_df)

        total_energy                                             forces  \
38214      -0.378737  [[-0.00143917, 0.00044284, 0.00022077], [0.001...   
38215      -0.376824  [[-0.00089992, 0.00026214, 0.00011855], [0.000...   
38217      -0.379179  [[0.00025588, -4.576e-05, -5.01e-05], [-0.0001...   
129159     -0.384045  [[-0.0, -0.0, 0.0], [-0.0, -0.0, 0.0], [-0.0, ...   
129160     -0.380679  [[-0.0, -0.0, 0.0], [-0.0, -0.0, 0.0], [-0.0, ...   
129162     -0.385480  [[-0.0, 0.0, 0.0], [-0.0, -0.0, 0.0], [-0.0, 0...   
156983      0.084970  [[-0.0, 0.00171141, 0.0], [-0.00148212, -0.000...   
156984      0.084960  [[-0.0, 0.00093048, 0.0], [-0.00080582, -0.000...   
156985      0.084967  [[-0.0, 0.00147545, 0.0], [-0.00127778, -0.000...   
160713      0.082212  [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [-0.0, -0.0...   
160714      0.079580  [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [-0.0, -0.0...   
160715      0.079635  [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [-0.0, -0.0...   
162008     -0.384044  [[0